# Hack My Ride
## Exploratory Data Analysis

* * *

The purpose of this notebook is to get a first sneak into the datasets provided, how it is structured, and get an understanding into how we are going to deal with the data as well as some initial analysis.

In [56]:
import pandas as pd
import geopandas as gpd
import json
import gtfs_kit as gk
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [6]:
data_folder = '/Users/alfredo.leon/Desktop/DataMiningProject/Project Data-20221021/'

### CSV Files
#### Agency

In [11]:
agency = pd.read_csv(data_folder + "gtfs3Sept/agency.txt", index_col=None)
agency.head()

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,STIB-MIVB,STIB,http://www.stib-mivb.be,Europe/Brussels,fr,070 23 2000


Only one row.

#### Calendar Dates

In [31]:
calendar_dates = pd.read_csv(data_folder + "gtfs3Sept/calendar_dates.txt", index_col='service_id')
print(calendar_dates.shape)
calendar_dates.sort_values(by="service_id").head()

(311, 2)


,date,exception_type
service_id,,
233898050,20210830,2
235005071,20210827,2
235536070,20210827,2
235811052,20210830,2
235954071,20210827,2


This table relates services with dates.
#### Calendar

In [35]:
calendar = pd.read_csv(data_folder + "gtfs3Sept/calendar.txt", index_col=None)
calendar.loc[calendar['service_id'] == 233898050]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
48,233898050,1,1,1,1,1,0,0,20210823,20210831


Apparently this table gives a more detailed description of the schedule of a particular service.
#### Routes

In [36]:
routes = pd.read_csv(data_folder + "gtfs3Sept/routes.txt", index_col='route_id')
routes.head()

,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
route_id,,,,,,,
2,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF
4,2,SIMONIS - ELISABETH,NaN,1,NaN,F57000,FFFFFF
5,3,ESPLANADE - CHURCHILL,NaN,0,NaN,B5BA05,000000
10,4,GARE DU NORD - STALLE (P),NaN,0,NaN,F25482,000000
1,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF


In [38]:
routes.loc[routes['route_short_name'] == '71']

,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
route_id,,,,,,,
65,71,DE BROUCKERE - DELTA,NaN,3,NaN,4C8B33,000000


This table describes the routes, provides an id for each route. Note that the route identifier differs from the route short name.
#### Shapes

In [17]:
shapes = pd.read_csv(data_folder + "gtfs3Sept/shapes.txt", index_col=None)
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,001m0042,50.841872,4.464541,10001
1,001m0042,50.843270,4.463607,10002
2,001m0042,50.843517,4.463443,10003
3,001m0042,50.843755,4.463283,10004
4,001m0042,50.844054,4.463082,10005


I guess this csv is to be able to put the shapes on a specific latitude and longitude.
#### Stop Times

In [39]:
stop_times = pd.read_csv(data_folder + "gtfs3Sept/stop_times.txt", index_col=None)
print(stop_times.shape)
stop_times.head()

(2820509, 7)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,112387248235954071,21:07:00,21:07:00,4014,1,0,0
1,112387248235954071,21:09:00,21:09:00,3231,2,0,0
2,112387248235954071,21:10:08,21:10:08,3232,3,0,0
3,112387248235954071,21:11:00,21:11:00,3233,4,0,0
4,112387248235954071,21:11:43,21:11:43,3239,5,0,0


This table records the trip stop times.
#### Stops

In [43]:
stops = pd.read_csv(data_folder + "gtfs3Sept/stops.txt", index_col=None)
print(stops.shape)
stops.loc[stops['stop_id']=='4014']

(2998, 10)


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1653,4014,NaN,BEEKKANT,NaN,50.854237,4.323407,NaN,NaN,0,NaN


This table gives more details on a certain stop. Provides latitud and longitude.
#### Translations

In [20]:
translations = pd.read_csv(data_folder + "gtfs3Sept/translations.txt", index_col=None)
translations.head()

,trans_id,translation,lang
0,STIB,STIB,fr
1,STIB,MIVB,nl
2,http://www.stib-mivb.be,http://www.stib-mivb.be/index.htm?l=fr,fr
3,http://www.stib-mivb.be,http://www.stib-mivb.be/index.htm?l=nl,nl
4,ABBAYE,Abbaye,fr


Apparently this kind of table is used to handle all the different languages used in the STIB.
#### Trips

In [115]:
trips = pd.read_csv(data_folder + "gtfs3Sept/trips.txt", index_col=None)
print(trips.shape)
trips.loc[trips['route_id']==65]

(135249, 7)


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
8216,65,237625071,112425365237625071,DE BROUCKERE,1,8946403,071b0195
8217,65,237625071,112425366237625071,DE BROUCKERE,1,8946390,071b0195
8218,65,237625071,112425367237625071,DE BROUCKERE,1,8946391,071b0195
8219,65,237625071,112425368237625071,DE BROUCKERE,1,8946398,071b0195
8220,65,237625071,112425369237625071,DE BROUCKERE,1,8946392,071b0195
...,...,...,...,...,...,...,...
68462,65,238249601,113492229238249601,DE BROUCKERE,1,8958163,071b0195
68463,65,238249601,113492230238249601,DE BROUCKERE,1,8958162,071b0195
68464,65,238249601,113496049238249601,DELTA,0,8958152,071b0208
68465,65,238249601,113496397238249601,PORTE DE NAMUR,1,8958430,071b0209


This table matches services, routes, trips and shapes.

In [51]:
path = Path(data_folder+"gtfs3Sept.zip")
feed = (gk.read_feed(path, dist_units='km'))
feed.validate()

,type,message,table,rows
0,warning,Feed expired,calendar/calendar_dates,[]


In [52]:
feed.map_stops(feed.stops.stop_id[:])

In [99]:
feed.map_routes(['65'])

In [116]:
feed.map_trips(trip_ids=['113496397238249601'])

### Understanding the JSONs

In [100]:
with open(data_folder+"vehiclePosition08.json", "r") as f:
    vehicle8 = json.load(f)

```
{      "lineId": "71",      "vehiclePositions": [        {          "directionId": "3520",          "distanceFromPoint": 395,          "pointId": "2397"        },        {          "directionId": "2596",          "distanceFromPoint": 0,          "pointId": "3521"        },        {          "directionId": "3520",          "distanceFromPoint": 54,          "pointId": "2296"        },        {          "directionId": "2596",          "distanceFromPoint": 66,          "pointId": "5612"        },        {          "directionId": "3520",          "distanceFromPoint": 0,          "pointId": "2596"        },        {          "directionId": "3520",          "distanceFromPoint": 0,          "pointId": "5611"        },        {          "directionId": "3520",          "distanceFromPoint": 185,          "pointId": "3510"        },        {          "directionId": "3520",          "distanceFromPoint": 298,          "pointId": "1597"        },        {          "directionId": "3520",          "distanceFromPoint": 38,          "pointId": "3503"        },        {          "directionId": "2596",          "distanceFromPoint": 34,          "pointId": "3556"        },        {          "directionId": "2596",          "distanceFromPoint": 34,          "pointId": "2963"        }      ]    },    {      "lineId": "72",      "vehiclePositions": [        {          "directionId": "5407",          "distanceFromPoint": 0,          "pointId": "3522"        }      ]    }
```

Using route 71 (Id 65) as an example, we can find that in the jsons it is treated as "lineId". Let's analyse some of the directionIds reported here.

In [124]:
feed.stops[stops['stop_id'].isin(['3520', '2596'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1027,2596,NaN,DE BROUCKERE,NaN,50.849654,4.352651,NaN,NaN,0,NaN
1541,3520,NaN,DELTA,NaN,50.816744,4.405058,NaN,NaN,0,NaN


Ok, apparently on the jsons, lineId will refer to the route_short_name attribute in the routes table, instead it should refer to route_id. So data map ends like this:
- lineId = routes.route_short_name
- directionId = stops.stop_id (Terminal stop)
- pointId = stops.stop_id (Last visited stop)
- distanceFromPoint = Distance in meters between bus and pointId (Last visited stop)

Next steps? Brainstorm ideas for project (since it's open ended):
- Select routes that bring students to ULB or close (95, 71, etc).
- Extract their recorded data from jsons.
- Generate maps across time